In [73]:
from pymongo import MongoClient

In [2]:
import urllib.parse

In [27]:
url = 'https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=10&isuri=1&7003=200&7035=-1&7004=naics&7005=-1&7006=00000&7036=-1&7001=1200&7002=1&7090=70&7007=-1&7093=levels'
params = urllib.parse.parse_qs(url.partition('?')[2])

In [28]:
params

{'reqid': ['70'],
 'step': ['10'],
 'isuri': ['1'],
 '7003': ['200'],
 '7035': ['-1'],
 '7004': ['naics'],
 '7005': ['-1'],
 '7006': ['00000'],
 '7036': ['-1'],
 '7001': ['1200'],
 '7002': ['1'],
 '7090': ['70'],
 '7007': ['-1'],
 '7093': ['levels']}

In [22]:
qs = urllib.parse.urlencode(params)

In [20]:
base_url = 'https://api.foo.bar/'

In [32]:
import requests
from bs4 import BeautifulSoup

In [33]:
name = "Elon Musk"
name = name.replace(' ', '_')
url = f"https://en.wikipedia.org/wiki/{name}"

In [34]:
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, 'html.parser')
sel = 'span.ForceAgeToShow'
soup.select_one(sel).text.partition('\xa0')[2].strip(')')

In [126]:
def get_age(name, scraper=scrape_mongo):
    """Return the age of the named celebrity via Wikipedia."""
    name = name.replace(' ', '_')
    url = f"https://en.wikipedia.org/wiki/{name}"
    html = scraper(url)
    soup = BeautifulSoup(html, 'html.parser')
    sel = 'span.ForceAgeToShow'
    return int(soup.select_one(sel).text.partition('\xa0')[2].strip(')'))

In [110]:
def scrape_mongo(url):
    """Return the HTML content of url."""
    mc = MongoClient()
    db = mc['cache']
    pages = db['pages']
    cached_page = pages.find_one({'url': url})
    if cached_page is None:
        print(f"Fetching {url}")
        response = requests.get(url)
        if response.status_code == 200:
            pages.insert_one({'url': url,
                              'html': response.content})   
        else:
            pages.insert_one({'url': url,
                              'html': None})   
    return pages.find_one({'url': url})['html']
    

In [111]:
html = scrape_mongo(url)

In [112]:
html[:300]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Elon Musk - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=windo'

In [129]:
mc = MongoClient()
db = mc['cache']
pages = db['pages']
cached_page = pages.find_one({'url': url})

In [114]:
cached_page is None

False

In [115]:
class Scraper:
    def __init__(self):
        self.cache = {}
    def scrape(self, url):
        if url not in self.cache:
            response = requests.get(url)
            if response.status_code == 200:
                self.cache[url] = response.content
            else:
                self.cache[url] = None
        return self.cache[url]

scraper = Scraper()

In [116]:
get_age('Jane Goodall')

84

In [117]:
get_age('Jane Goodall', scraper=scraper.scrape)

84

In [118]:
get_age('Guido Van Rossum')

62

In [119]:
names = ['Linus Torvalds',
         'Britney Spears',
         'Noam Chomsky',
         'Ric Flair']

In [104]:
from multiprocessing import Pool

In [105]:
pool = Pool(20)

In [128]:
pool.map(get_age, names)

[48, 36, 89, 69]

In [123]:
list(map(get_age, names))

[48, 36, 89, 69]

In [120]:
db['pages'].find().count()

7

In [125]:
get_age('Guido van Rossum')

62